<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       MInfo Function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:20px;font-family:Arial'><b>MInfo</b></p>
<p style = 'font-size:16px;font-family:Arial'>MInfo() returns one row for each matrix instance found in the target table. The summary provides a high-level understanding of the matrix data as well as determining if there is an issue. For example, if you expect data to have a certain number of rows and columns, a certain data type, or only a specific range of values, then checking the summary information can verify it. Each returned row provides the following information about a matrix:</p>
<li style = 'font-size:16px;font-family:Arial'>Row index data type</li>
<li style = 'font-size:16px;font-family:Arial'>Starting row index value</li>
<li style = 'font-size:16px;font-family:Arial'>Ending row index value</li>
<li style = 'font-size:16px;font-family:Arial'>Number of row index entries</li>
<li style = 'font-size:16px;font-family:Arial'>Indicator that the matrix is regular (discrete) or irregular along row index</li>
<li style = 'font-size:16px;font-family:Arial'>Row index sample interval for regular series or average sample interval for irregular series</li>
<li style = 'font-size:16px;font-family:Arial'>Column index data type</li>
<li style = 'font-size:16px;font-family:Arial'>Starting column index value</li>
<li style = 'font-size:16px;font-family:Arial'>Ending column index value</li>
<li style = 'font-size:16px;font-family:Arial'>Number of column index entries</li>
<li style = 'font-size:16px;font-family:Arial'>Indicator that the matrix is regular (discrete) or irregular along column index</li>
<li style = 'font-size:16px;font-family:Arial'>Column index sample interval for regular series or average sample interval for irregular series</li>
<li style = 'font-size:16px;font-family:Arial'>Content type</li>
<li style = 'font-size:16px;font-family:Arial'>Minimum sample magnitude</li>
<li style = 'font-size:16px;font-family:Arial'>Maximum sample magnitude</li>
<li style = 'font-size:16px;font-family:Arial'>Average sample magnitude</li>
<li style = 'font-size:16px;font-family:Arial'>Number of NULL values</li>
<li style = 'font-size:16px;font-family:Arial'>Indicator matrix is well formed or malformed</li></p>

</p>


<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDMatrix,
    MInfo,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_MInfo.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", ["ocean_buoys2"])

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
df = DataFrame.from_table("ocean_buoys2")
df

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. MInfo</b>
<p style = 'font-size:16px;font-family:Arial'>MInfo() displays a result set containing metadata about each matrix. The function returns one row per matrix processed. In addition, each input payload produces the output variables for MIN_MAG, MAX_MAG and AVG_MAG. The output varies depending on the number of input payloads.</p>

<p style = 'font-size:16px;font-family:Arial'>Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(MInfo)

<p style = 'font-size:16px;font-family:Arial'>The first step is to convert the data into TDSeries, which is required for the input time series which are passed to the MInfo function.</p>

In [ ]:
res = TDMatrix(data=df,
                   id=['ocean_name', 'buoyid'],
                   row_index='TD_TIMECODE',
                   column_index = 'space',
                   row_index_style="TIMECODE",
                   column_index_style="SEQUENCE",
                   payload_field='jsoncol.Measure.salinity',
                   payload_content='REAL')

In [ ]:
uaf_out = MInfo(data=res)

minfo_df = uaf_out.result
minfo_df

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables created above.</p>

In [ ]:
db_drop_table("ocean_buoys2")

In [ ]:
remove_context()

<hr style="height:1px;border:none;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>MInfo function reference: <a href = 'https://docs.teradata.com/search/all?query=MInfo&content-lang=en-US'>here</a></li>
    
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>